# Built-in Modules

Python takes a "batteries included" approach to the standard library. Many other languages ship with a small number of common packages and require you to look elsewhere for important functionality. Although Python also has an impressive repository of community-built modules, it strives to provide, in tis default installation, the most important modules, for common uses of the language.

The full set of standard modules is too large to cover in this book. But some of these built-in packages are so closely interwined with idiomatic Python that they may as well be part of the language specification. These essential built-iin modules are especially important when writing the intricate, error-prone parts of programs.

# I42 : Define Function Decorators with functools.wraps

- Python has special syntax for *decorators* taht can be applied to functions. Decorators have the ability to run additional code before and after any calls to the functions they wrap. This allows them to access and modify input arguments and return values. This functioinality can be useful for enforcing semantics, debuggin, registering functions, and more.

- For example, say you want to print the arguments and return value of a function call. This is especially helpful when debugging a stack of function calls from a recursive function. Here, I define such a decorator:

In [1]:
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r' %
             (func.__name__, args, kwargs, result))
        return result
    return wrapper

- I can apply this to a function using the @ symbol

In [2]:
@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n-2) + fibonacci(n-1))

- The @ symbol is equivalent to calling the decorator on the function it wraps and assigning the return value to the original name in the same scope.

In [3]:
fibonacci = trace(fibonacci)

- Callling this decorated function will run the wrapper code before and after fibonacci runs, printing the arguments and return value at each level in the recursive stack.

In [4]:
fibonacci(3)

fibonacci((1,), {}) -> 1
wrapper((1,), {}) -> 1
fibonacci((0,), {}) -> 0
wrapper((0,), {}) -> 0
fibonacci((1,), {}) -> 1
wrapper((1,), {}) -> 1
fibonacci((2,), {}) -> 1
wrapper((2,), {}) -> 1
fibonacci((3,), {}) -> 2
wrapper((3,), {}) -> 2


2

- This works well, but it has an unintended side effect. The value returned by the decorator - the function that's called above - doesn't think it's named fibonacci.

In [5]:
print(fibonacci)

<function trace.<locals>.wrapper at 0x7f01c5301bf8>


- The cause of this isn't hard to see. The trace function returns the wrapper it defines. The wrapper function is what's assigned to the fibonacci name in the containing module because of the decorator. This behavior is problematic because it undermines tools that do introspection, such as debuggers and object serializers.

- For example, the help built-in function is useless on the decorated fibonacci function.

In [7]:
help(fibonacci)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



- The solution is to use the wraps helper function from the functools built-in module. This is a decorator that helps you write decorators. Applying it to the wrapper function will copy all of the important metadata about the inner function to the outer function.

In [13]:
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        #...
        pass
    return wrapper

@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n-2) + fibonacci(n-1))

- Now, running the help function produces the expected result, even though the function is decorated.

In [14]:
help(fibonacci)

Help on function fibonacci in module __main__:

fibonacci(n)
    Return the n-th Fibonacci number



- Calling help is just one example of how decorators can subtly cause problems. Python functions have many other standard attributes that must be preserved to maintain the interface of functions in the language. Using wraps ensures that you'll always get the correct behavior.

## Things to Remember
- Decorators are Python syntax for allowing function to modify another function at runtime.
- Using decorators can cause strange behaviors in tools that do introspection, such as debuggers.
- Use the wraps decorator from the functools built-in module when you define your own decorators to avoid any issues.

# I43 : Consider contextlib and with statements for Reusable try/finally Behavior


- The with statement in Python is used to indicate when code is running in a special context. For example, mutual exclusion locks can be used in with statements to indicate that the indented code only runs while the lock is held.

In [15]:
from threading import Lock
lock = Lock()
with lock:
    print('Lock is held')

Lock is held


- The example above is equivalent to this try/finally construction because the Lock class properly enables the with statement.

In [16]:
lock.acquire()
try:
    print('Lock is held')
finally:
    lock.release()

Lock is held


- The with statement version of this is better because it eliminates the need to write the repetitive code of the try/finally construction. It's easy to make your objects and functions capable of use in with statements by using the contextlib built-in module. This module contains the contextmanager decorator, which lets a simple function be used in with satetements. This is much easier than defining a new class with the special methods \_\_enter\_\_ and \_\_exit\_\_.

- For example, say you want a region of your code to have more debug logging sometimes. Here, I define a function that does logging at two severity levels:

In [19]:
import logging

def my_function():
    logging.debug('Some debug data')
    logging.error('Error log here')
    logging.debug('More debug data')

- The default log level for my program is WARNING, so only the error message will print to screen when I run the function.

In [20]:
my_function()

ERROR:root:Error log here


- I can elevate the log level of this function temporarily by defining a context manager. This helper function boosts the logging severity level before running the code in the with block and reduces the logging severity lebel afterward.

In [25]:
from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield
    finally:
        logger.setLevel(old_level)

- The yield expression is the point at which the with block's contents will execute. Any exceptions that happen in the with block will be re-raised by the yield expression for you to catch in the helper function.

- Now, I can call the same logging function again, but in the debug\_logging context. This time, all of the debug messages are printed to the screen during the with block. The same function running outside the with block won't print debug messages.

In [26]:
with debug_logging(logging.DEBUG):
    print('Inside:')
    my_function()
print('After:')
my_function()

DEBUG:root:Some debug data
ERROR:root:Error log here
DEBUG:root:More debug data
ERROR:root:Error log here


Inside:
After:


** Using with Targets **

- The context manager passed to a with statement may also return an object. This object is assigned to a local variable in the as part of the compound statement. This gives the code running in the with block the ability to directly interact with its context.

- For example, say you want to write a file and ensure that it's always closed correctly. You can do this by passing open to the with statement. Open returns a file handle for the as target of with and will close the handle when the with block exits.

In [27]:
with open('/tmp/my_output.txt', 'w') as handle:
    handle.write('This is some data!')

- This approach is preferable to manually opening and closing the file handle every time. It gives you confidence that the file is eventually closed when execution leaves the with statement. It also encourages you to reduce the amount of code that executes while the file handle is open, which is good practice in general.

- To enable your own functions to supply values for as targets, all you need to do is yield a value from your context mangaer. For example, here I define a context manager to fetch a Logger instance, set its level, and then yield it for the as target.

In [28]:
@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)

- Calling logging methods like debug on the as target will produce output because the logging severity level is set low enough in the with block. Using the logging module directly won't print anything becuase the default logging severity level for the default program logger is WARNING.

In [29]:
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug('This is my message!')
    logging.debug('This will not print')

DEBUG:my-log:This is my message!


- After the with statement exits, calling debug logging methods on the Logger named 'my-log' will not print anyhing because the default logging severity level has been restored. Error log messages will always print.

In [30]:
logger = logging.getLogger('my-log')
logger.debug('Debug will not print')
logger.error('Error will print')

ERROR:my-log:Error will print


## Things to Remember 
- The with statement allows you to reuse logic from try/finally blocks and reduce visual noise.
- The contextlib built-in module provides a contextmanager decorator that makes it easy to use your own functions in with statements.
- The value yielded by context managers is supplied to the as part of the with statement. It's useful for letting your code directly access the cause of the special context.

# I44 : make pickle Reliable with copyreg